In [22]:
import pandas as pd 
import json 

In [ ]:
data = pd.read_csv('../res/sample_taxi_region.csv') 

In [23]:
with open('../res/sz_block_mars.json', 'r') as f : 
    block_data = json.load(f) 

In [24]:
block_data = block_data['features']

In [25]:
def in_this_region(coordiate, region) : 
    cross_cnt = 0 
    for i in range(len(region)) : 
        l, r = region[i-1], region[i] 
        if l[1] == r[1] : 
            continue 
        if (l[1] > coordiate[1]) == (r[1] > coordiate[1]): 
            continue 
        x = (coordiate[1] - l[1]) * (r[0] - l[0]) / (r[1] - l[1]) + l[0] 
        if x < coordiate[0]: 
            continue 
        lx = l[0] if l[0] < r[0] else r[0] 
        rx = l[0] if l[0] > r[0] else r[0] 
        if lx <= x <= rx : 
            pass 
        else: 
            continue 
        cross_cnt += 1 
    return cross_cnt % 2 == 1 

def in_which_block(coordinate) : 
    results = [] 
    for i in range(len(block_data)) : 
        regions = block_data[i]['geometry']['coordinates'] 
        for region in regions: 
            is_in_this_region = in_this_region(coordinate, region) 
            if is_in_this_region : 
                results.append((block_data[i]['properties']['NAME'], block_data[i]['properties']['DISTRICT'], block_data[i]['properties']['BLOCK_ID']))
                break 
    return results 

In [35]:
# Config of separate work 

TOTAL = 1000
IAM = 0 # 0, 1, ... , TOTAL-1 

In [36]:
# split data average as TOTAL parts 
data_len = len(data) 
split_len = data_len // TOTAL 
my_data = data.iloc[split_len * IAM : split_len * (IAM + 1)] 
print ('workload: ', len(my_data))

workload:  9843


In [37]:
# split data by block 
block_id = [-1 for _ in range(len(my_data))]
lon = my_data['lon'] 
lat = my_data['lat'] 
index = 0
for loni, lati in zip(lon, lat) : 
    i = in_which_block([loni, lati]) 
    if len(i) == 1: 
        block_id[index] = i[0][2] 
    index += 1 